# Compilation Examples with Routing

In [1]:
from pytket import Circuit
from pytket.circuit.display import view_browser as vb
from pytket.extensions.aqt.backends.aqt_multi_zone import AQTMultiZoneBackend
from pytket.extensions.aqt.multi_zone_architecture.named_architectures import (
    four_zones_in_a_line,
    racetrack,
    grid12
)
from pytket.extensions.aqt.multi_zone_architecture.initial_placement.settings import (
    InitialPlacementSettings,
    InitialPlacementAlg,
)
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.settings import (
    RoutingSettings,
    RoutingAlg,
)
from pytket.extensions.aqt.multi_zone_architecture.compilation_settings import (
    CompilationSettings,
)
import time

First lets define some circuits

In [2]:
## A simple GHZ circuit
ghz_circuit = Circuit(16)
ghz_circuit.H(0)
for i in range(ghz_circuit.n_qubits - 1):
    ghz_circuit.CX(i, i + 1)
ghz_circuit.measure_all()

## Quantum advantage circuit template
def quantum_advantage_circuit(Lx: int, Ly: int) -> Circuit:
    N = Lx * Ly
    Ntrot = 30
    dt = 0.2
    hx = 2
    J = 1

    def c(x, y):
        if (x - y) % N == Lx or (y - x) % N == Lx:
            return 1
        if x // Lx == y // Lx and x == y - (y % Lx) + (((y % Lx) + 1) % Lx):
            return 1
        if x // Lx == y // Lx and y == x - (x % Lx) + (((x % Lx) + 1) % Lx):
            return 1
        return 0

    coupling_list = []
    for i in range(0, N):
        for j in range(i + 1, N):
            if c(i, j) == 1:
                coupling_list.append([i, j])

    U = Circuit(N)

    for t in range(Ntrot):
        n_trotter_steps = t

        if n_trotter_steps == 0:
            for j in range(N):
                U.Rx(-2 * dt * hx / 2, j)

        for coupling in coupling_list:
            U.ZZPhase(-2 * dt * J, coupling[0], coupling[1])
        for j in range(N):
            U.Rx(-2 * dt * hx, j)

        if n_trotter_steps == Ntrot - 1:
            for j in range(N):
                U.Rx(2 * dt * hx / 2, j)
    return U

## 56 qubit quantum advantage circuit
advantage_circuit_56 = quantum_advantage_circuit(7, 8)

## 30 qubit quantum advantage circuit
advantage_circuit_30 = quantum_advantage_circuit(5, 6)



Now some MultiZone Backends

In [6]:
line_backend = AQTMultiZoneBackend(
    architecture=four_zones_in_a_line, access_token="invalid"
)

racetrack_backend = AQTMultiZoneBackend(
    architecture=racetrack, access_token="invalid"
)

grid_backend = AQTMultiZoneBackend(
    architecture=grid12, access_token="invalid"
)



In [3]:
## 30 qubit quantum advantage circuit
def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(0, n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

nnn = 16
qft_circ = build_qft_circuit(nnn)
vb(qft_circ)

In [15]:
circ_tmp = Circuit(9, name="bla")
circ_tmp.ZZMax(0,1).ZZMax(2,3).ZZMax(4,5).ZZMax(7,8)
circ_tmp.ZZMax(1,2).ZZMax(2,3).ZZMax(5,6).ZZMax(6,7).ZZMax(3,4)
vb(circ_tmp)

And some different compilation settings

In [4]:
graph_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.graph_partition,
    zone_free_space=2,
    n_threads=1,
    max_depth=200,
)
order_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.qubit_order,
    zone_free_space=2,
    n_threads=1,
    max_depth=200,
)
graph_routing = RoutingSettings(
    algorithm=RoutingAlg.graph_partition,
    debug_level=1,
    n_threads=1,
)
graph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=graph_routing,
)

greedy_routing = RoutingSettings(
    algorithm=RoutingAlg.greedy,
)

greedy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=greedy_routing,
)

In [7]:
ghz_precompiled = line_backend.precompile_circuit(ghz_circuit, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(ghz_precompiled, graph_compilation_settings)
ghz_greedy_routed = line_backend.route_precompiled(ghz_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
Depth List:
[(3, 4)]
[(4, 5)]
[(5, 6)]
[(6, 7)]
cut_cost:  2751977
-------
Z0: [0, 1, 2, 3] -> [0, 1, 2, 3, 4] -- (+4)
Z1: [4, 5, 6, 7, 8, 9] -> [5, 6, 7, 8, 9] -- (-4)
Z2: [10, 11, 12, 13, 14, 15] -> [10, 11, 12, 13, 14, 15] -- ()
Z3: [] -> [] -- ()
Depth List:
[(4, 5)]
[(5, 6)]
[(6, 7)]
[(7, 8)]
cut_cost:  2777160
-------
Z0: [0, 1, 2, 3, 4] -> [0, 2, 3, 4, 5] -- (+5, -1)
Z1: [5, 6, 7, 8, 9] -> [1, 6, 7, 8, 9] -- (+1, -5)
Z2: [10, 11, 12, 13, 14, 15] -> [10, 11, 12, 13, 14, 15] -- ()
Z3: [] -> [] -- ()
Depth List:
[(5, 6)]
[(6, 7)]
[(7, 8)]
[(8, 9)]
cut_cost:  2777236
-------
Z0: [0, 2, 3, 4, 5] -> [0, 3, 4, 5, 6] -- (+6, -2)
Z1: [1, 6, 7, 8, 9] -> [1, 2, 7, 8, 9] -- (+2, -6)
Z2: [10, 11, 12, 13, 14, 15] -> [10, 11, 12, 13, 14, 15] -- ()
Z3: [] -> [] -- ()
Depth List:
[(6, 7)]
[(7, 8)]
[(8, 9)]
[(9, 10)]
cut_cost:  2777798
-------
Z0: [0, 3, 4, 5, 6] -> [0, 4, 5, 6, 7] -- (+7, -3)
Z1: [1, 2, 7, 8, 9] -> [1, 2, 3, 8

In [8]:
qft_precompiled = line_backend.precompile_circuit(qft_circ, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(qft_precompiled, graph_compilation_settings)
ghz_greedy_routed = line_backend.route_precompiled(qft_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
Depth List:
[(0, 4), (0, 4)]
[(1, 4), (0, 5), (1, 4), (0, 5)]
[(2, 4), (1, 5), (2, 4), (0, 6), (1, 5), (0, 6)]
[(3, 4), (2, 5), (3, 4), (1, 6), (2, 5), (0, 7), (1, 6), (0, 7)]
cut_cost:  3335841
-------
Z0: [0, 1, 2, 3] -> [0, 1, 2, 3, 4] -- (+4)
Z1: [4, 5, 6, 7, 8, 9] -> [5, 6, 7, 8, 9] -- (-4)
Z2: [10, 11, 12, 13, 14, 15] -> [10, 11, 12, 13, 14, 15] -- ()
Z3: [] -> [] -- ()
Depth List:
[(0, 5), (0, 5)]
[(1, 5), (0, 6), (1, 5), (0, 6)]
[(2, 5), (1, 6), (2, 5), (0, 7), (1, 6), (0, 7)]
[(3, 5), (2, 6), (3, 5), (1, 7), (2, 6), (0, 8), (1, 7), (0, 8)]
cut_cost:  3361158
-------
Z0: [0, 1, 2, 3, 4] -> [0, 1, 2, 3, 5] -- (+5, -4)
Z1: [5, 6, 7, 8, 9] -> [4, 6, 7, 8, 9] -- (+4, -5)
Z2: [10, 11, 12, 13, 14, 15] -> [10, 11, 12, 13, 14, 15] -- ()
Z3: [] -> [] -- ()
Depth List:
[(0, 6), (0, 6), (4, 5), (4, 5)]
[(1, 6), (0, 7), (1, 6), (0, 7)]
[(2, 6), (1, 7), (2, 6), (0, 8), (1, 7), (0, 8)]
[(3, 6), (2, 7), (3, 6), (1, 8), (2, 

In [7]:

adv_precomp = racetrack_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = racetrack_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = racetrack_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
Depth List:
[(0, 7), (1, 8), (0, 7), (2, 3), (1, 8), (2, 3)]
[(0, 49), (1, 50), (7, 8), (0, 49), (2, 9), (1, 50), (3, 4), (7, 8), (2, 9), (3, 4)]
[(2, 51), (8, 9), (0, 1), (3, 10), (2, 51), (4, 5), (8, 9), (0, 1), (3, 10), (4, 5)]
[(8, 15), (3, 52), (9, 10), (1, 2), (4, 11), (8, 15), (3, 52), (5, 6), (9, 10), (1, 2), (4, 11), (5, 6)]
cut_cost:  12714666
-------
Z0: [0, 1, 2, 6] -> [2, 3, 6] -- (+3, -0, -1)
Z1: [3, 7, 49, 50] -> [0, 7, 49, 50] -- (+0, -3)
Z2: [10, 11, 12, 17] -> [10, 11, 12, 17] -- ()
Z3: [13, 18, 19, 20] -> [13, 18, 19, 20] -- ()
Z4: [] -> [] -- ()
Z5: [] -> [] -- ()
Z6: [] -> [1, 8] -- (+8, +1)
Z7: [] -> [] -- ()
Z8: [] -> [] -- ()
Z9: [] -> [] -- ()
Z10: [] -> [] -- ()
Z11: [] -> [] -- ()
Z12: [] -> [] -- ()
Z13: [] -> [] -- ()
Z14: [] -> [] -- ()
Z15: [] -> [] -- ()
Z16: [] -> [] -- ()
Z17: [] -> [] -- ()
Z18: [38, 39, 45, 46] -> [38, 39, 45, 46] -- ()
Z19: [36, 37, 43, 44] -> [36, 37, 43,

In [8]:

adv_precomp = grid_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = grid_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = grid_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Depth List:
[(1, 6), (1, 6), (0, 25), (0, 25), (2, 7), (2, 7), (3, 8), (4, 9), (3, 8), (4, 9)]
[(1, 26), (5, 6), (1, 26), (5, 6), (2, 27), (2, 27), (3, 28), (4, 29), (3, 28), (4, 29)]
[(6, 7), (0, 1), (6, 7), (0, 1), (5, 9), (5, 9)]
[(6, 11), (7, 8), (1, 2), (6, 11), (7, 8), (1, 2), (5, 10), (0, 4), (5, 10), (0, 4)]
cut_cost:  13690837
-------
Z0: [] -> [] -- ()
Z1: [] -> [] -- ()
Z2: [] -> [] -- ()
Z3: [0, 1, 2, 3, 4, 5] -> [0, 1, 3, 5, 6, 8, 25] -- (+6, +8, +25, -2, -4)
Z4: [] -> [24] -- (+24)
Z5: [9, 10, 11, 12, 28, 29] -> [4, 9, 10, 11, 12, 28, 29] -- (+4)
Z6: [13, 14, 18, 19, 23, 24] -> [2, 7, 13, 14, 18, 19, 23] -- (+2, +7, -24)
Z7: [15, 16, 17, 20, 21, 22] -> [15, 16, 17, 20, 21, 22] -- ()
Z8: [6, 7, 8, 25, 26, 27] -> [26, 27] -- (-6, -7, -8, -25)
Z9: [] -> [] -- ()
Z10: [] -> [] -- ()
Z11: [] -> [] -- ()
Depth List:
[(1, 26), (1, 26), (2, 27), (2, 27), (3, 28), (3, 28), (6, 7), (6, 7), (5, 9), (5, 9)]
